<center> 
    <h1> Transformer TTS: A Text-to-Speech Transformer in TensorFlow 2 </h1>
    <h2> Audio synthesis with Forward Transformer TTS and MelGAN Vocoder</h2>
</center>

## Forward Model

In [ ]:
# Clone the Transformer TTS and WaveRNN repos
!git clone https://github.com/as-ideas/TransformerTTS.git
!git clone https://github.com/seungwonpark/melgan.git

In [ ]:
# Install requirements
!apt-get install -y espeak
!pip install -r TransformerTTS/requirements.txt

In [ ]:
# Download the pre-trained weights
! wget https://public-asai-dl-models.s3.eu-central-1.amazonaws.com/TransformerTTS/ljspeech_melgan_forward_transformer.zip
! unzip ljspeech_melgan_forward_transformer.zip

In [ ]:
# Set up the paths
from pathlib import Path
MelGAN_path = 'melgan/'
TTS_path = 'TransformerTTS/'
config_path = Path('ljspeech_melgan_forward_transformer/melgan')

import sys
sys.path.append(TTS_path)

In [ ]:
# Load pretrained models
from utils.config_manager import ConfigManager
from utils.audio import Audio

import IPython.display as ipd

config_loader = ConfigManager(str(config_path), model_kind='forward')
audio = Audio(config_loader.config)
model = config_loader.load_model(str(config_path / 'forward_weights/ckpt-93'))

In [ ]:
# Synthesize text
sentence = 'Scientists at the CERN laboratory, say they have discovered a new particle.'
out = model.predict(sentence)

In [ ]:
# Convert spectrogram to wav (with griffin lim)
wav = audio.reconstruct_waveform(out['mel'].numpy().T)
ipd.display(ipd.Audio(wav, rate=config_loader.config['sampling_rate']))

You can also vary the speech speed

In [ ]:
# 20% faster
sentence = 'Scientists at the CERN laboratory, say they have discovered a new particle.'
out = model.predict(sentence, speed_regulator=1.20)
wav = audio.reconstruct_waveform(out['mel'].numpy().T)
ipd.display(ipd.Audio(wav, rate=config_loader.config['sampling_rate']))

In [ ]:
# 10% slower
sentence = 'Scientists at the CERN laboratory, say they have discovered a new particle.'
out = model.predict(sentence, speed_regulator=.9)
wav = audio.reconstruct_waveform(out['mel'].numpy().T)
ipd.display(ipd.Audio(wav, rate=config_loader.config['sampling_rate']))

### MelGAN

In [ ]:
# Do some sys cleaning
sys.path.remove(TTS_path)
sys.modules.pop('model')

In [ ]:
sys.path.append(MelGAN_path)
import torch
import numpy as np

vocoder = torch.hub.load('seungwonpark/melgan', 'melgan')
vocoder.eval()

mel = torch.tensor(out['mel'].numpy().T[np.newaxis,:,:])

In [ ]:
if torch.cuda.is_available():
    vocoder = vocoder.cuda()
    mel = mel.cuda()

with torch.no_grad():
    audio = vocoder.inference(mel)

In [ ]:
# Display audio
ipd.display(ipd.Audio(audio.cpu().numpy(), rate=22050))